# Clamp Quickstart Guide

This notebook demonstrates how to use Clamp for version control of RAG vector databases.

Clamp provides Git-like version control for document collections in vector databases, enabling:
- 🔄 Version tracking with commit messages
- ⏮️ Rollback to previous versions
- 📜 History viewing
- 🎯 Version-aware queries

## Setup

First, install the required packages if you haven't already:

```bash
pip install qdrant-client
pip install -e .
```

In [ ]:
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams
from clamp import ClampClient
import tempfile

## Initialize Clients

Create a Qdrant client (in-memory for this demo) and wrap it with ClampClient:

In [ ]:
# Create Qdrant client (in-memory for demo)
qdrant = QdrantClient(":memory:")

# Create a collection
collection_name = "demo_docs"
qdrant.create_collection(
    collection_name=collection_name,
    vectors_config=VectorParams(size=3, distance=Distance.COSINE),
)

# Initialize Clamp client
# Use a temporary database for this demo
db_path = tempfile.mktemp(suffix=".db")
clamp = ClampClient(qdrant, control_plane_path=db_path)

print(f"✅ Initialized Clamp with database at {db_path}")

## Version 1: Initial Document Set

Let's ingest our first version of FAQ documents:

In [ ]:
# Version 1: Initial FAQ documents
documents_v1 = [
    {
        "id": "faq1",
        "vector": [0.1, 0.2, 0.3],
        "question": "How do I reset my password?",
        "answer": "Click the 'Forgot Password' link on the login page.",
        "category": "account"
    },
    {
        "id": "faq2",
        "vector": [0.4, 0.5, 0.6],
        "question": "What are your business hours?",
        "answer": "We're open Monday to Friday, 9 AM to 5 PM EST.",
        "category": "general"
    },
]

commit_v1 = clamp.ingest(
    collection=collection_name,
    group="faq",
    documents=documents_v1,
    message="Initial FAQ documentation",
    author="alice"
)

print(f"✅ Committed version 1: {commit_v1[:8]}")

## Query Version 1

Use the active filter to query only the current version:

In [ ]:
# Get the filter for active documents
active_filter = clamp.get_active_filter("faq")

# Query using the filter
results = qdrant.scroll(
    collection_name=collection_name,
    scroll_filter=active_filter,
    limit=10
)

print(f"\n📚 Active documents (Version 1):")
for point in results[0]:
    print(f"  - {point.payload['question']}")
    print(f"    Answer: {point.payload['answer']}")
    print(f"    Version: {point.payload['__clamp_ver'][:8]}")
    print()

## Version 2: Updated Documents

Let's update our FAQ with improved answers and add a new question:

In [ ]:
# Version 2: Updated FAQ with better answers
documents_v2 = [
    {
        "id": "faq1_v2",
        "vector": [0.1, 0.2, 0.3],
        "question": "How do I reset my password?",
        "answer": "Click 'Forgot Password' on the login page. You'll receive a secure reset link via email within 5 minutes.",
        "category": "account"
    },
    {
        "id": "faq2_v2",
        "vector": [0.4, 0.5, 0.6],
        "question": "What are your business hours?",
        "answer": "We now offer 24/7 customer support! Contact us anytime.",
        "category": "general"
    },
    {
        "id": "faq3_v2",
        "vector": [0.7, 0.8, 0.9],
        "question": "How do I contact support?",
        "answer": "Email support@example.com, call 1-800-SUPPORT, or use our live chat.",
        "category": "support"
    },
]

commit_v2 = clamp.ingest(
    collection=collection_name,
    group="faq",
    documents=documents_v2,
    message="Updated FAQ with 24/7 support info and new contact question",
    author="bob"
)

print(f"✅ Committed version 2: {commit_v2[:8]}")

## Query Version 2

Notice that queries now automatically return the new version:

In [ ]:
# Query again - should get v2 documents
results_v2 = qdrant.scroll(
    collection_name=collection_name,
    scroll_filter=clamp.get_active_filter("faq"),
    limit=10
)

print(f"\n📚 Active documents (Version 2):")
for point in results_v2[0]:
    print(f"  - {point.payload['question']}")
    print(f"    Answer: {point.payload['answer']}")
    print(f"    Version: {point.payload['__clamp_ver'][:8]}")
    print()

print(f"Total active documents: {len(results_v2[0])}")

## View History

See all commits for a document group:

In [ ]:
# Get commit history
history = clamp.history("faq")

print("\n📜 Commit History:")
print()
for i, commit in enumerate(history):
    from datetime import datetime
    timestamp = datetime.fromtimestamp(commit.timestamp).strftime('%Y-%m-%d %H:%M:%S')
    
    marker = "*" if i == 0 else " "
    status = "(ACTIVE)" if i == 0 else ""
    
    print(f"{marker} Commit: {commit.hash[:8]}")
    print(f"  Author: {commit.author}")
    print(f"  Date:   {timestamp}")
    print(f"  Message: {commit.message} {status}")
    print()

## Check Status

Get detailed information about the current deployment:

In [ ]:
status = clamp.status(collection_name, "faq")

print("\n📊 Current Status:")
print(f"  Group: {status['group']}")
print(f"  Active Commit: {status['active_commit_short']}")
print(f"  Message: {status['message']}")
print(f"  Author: {status['author']}")
print(f"  Active Vectors: {status['active_count']}")
print(f"  Total Vectors: {status['total_count']}")

## Rollback to Version 1

Oops! Maybe the 24/7 support wasn't ready. Let's rollback:

In [ ]:
# Rollback to version 1
print(f"🔄 Rolling back to version 1...\n")

clamp.rollback(
    collection=collection_name,
    group="faq",
    commit_hash=commit_v1
)

## Verify Rollback

Query again to confirm we're back to version 1:

In [ ]:
# Query after rollback
results_after_rollback = qdrant.scroll(
    collection_name=collection_name,
    scroll_filter=clamp.get_active_filter("faq"),
    limit=10
)

print(f"\n📚 Active documents after rollback:")
for point in results_after_rollback[0]:
    print(f"  - {point.payload['question']}")
    print(f"    Answer: {point.payload['answer']}")
    print(f"    Version: {point.payload['__clamp_ver'][:8]}")
    print()

print(f"Total active documents: {len(results_after_rollback[0])}")
print(f"\n✅ Successfully rolled back to version 1!")

## Vector Search with Version Control

Demonstrate that search operations respect the active version:

In [ ]:
# Search with version filter
query_vector = [0.4, 0.5, 0.6]  # Similar to "business hours" question

search_results = qdrant.search(
    collection_name=collection_name,
    query_vector=query_vector,
    query_filter=clamp.get_active_filter("faq"),
    limit=3
)

print("\n🔍 Search Results:")
for result in search_results:
    print(f"  Score: {result.score:.4f}")
    print(f"  Question: {result.payload['question']}")
    print(f"  Answer: {result.payload['answer']}")
    print(f"  Version: {result.payload['__clamp_ver'][:8]}")
    print()

## Summary

In this quickstart, we:

1. ✅ Initialized Clamp with a Qdrant client
2. ✅ Ingested two versions of FAQ documents
3. ✅ Queried documents with version-aware filters
4. ✅ Viewed commit history
5. ✅ Rolled back to a previous version
6. ✅ Performed vector search with version control

### Key Takeaways

- **Automatic Version Management**: Clamp automatically injects metadata into your documents
- **Zero-Downtime Rollback**: Switch between versions instantly
- **Query Transparency**: Use `get_active_filter()` to ensure queries only hit active versions
- **History Tracking**: Full audit trail of all document changes

### Next Steps

- Try the CLI: `clamp history faq --collection demo_docs`
- Explore multiple document groups
- Integrate with your RAG pipeline
- Set up automated version testing

## Cleanup (Optional)

Remove the temporary database:

In [ ]:
import os

if os.path.exists(db_path):
    os.remove(db_path)
    print(f"🗑️  Cleaned up temporary database")